Training notebook:
https://www.kaggle.com/mostafaibrahim17/yolov5-vinbigdata

Credits:
https://www.kaggle.com/awsaf49/vinbigdata-2-class-filter

https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer

https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import shutil

In [ ]:
# load yolo submission
yolo = pd.read_csv('../input/yolo169/submission_2.csv')
effnetb6 = pd.read_csv('../input/vinbigdata-2class-prediction/2-cls test pred.csv') # AUC:0.98

In [ ]:
yolo.head()

In [ ]:
effnetb6.head()

In [ ]:
pred = pd.merge(yolo, effnetb6, on = 'image_id', how = 'left')
pred.head()

In [ ]:
low_thr  = 0.08
high_thr = 0.95

In [ ]:
def filter_2cls(row, low_thr=low_thr, high_thr=high_thr):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [ ]:
sub = pred.apply(filter_2cls, axis=1)
sub.head()

In [ ]:
sub[['image_id', 'PredictionString']].to_csv('submission.csv',index = False)